In [1]:
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import hub
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchaudio

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, recall_score, precision_score, balanced_accuracy_score, accuracy_score, classification_report
from sklearn.utils import shuffle

import scipy

from tqdm import tqdm

from datasets import load_dataset, Dataset, Audio
import librosa
from transformers import AutoFeatureExtractor, Wav2Vec2ForSequenceClassification

from models.basic_transformer import BasicTransformer

from src.utils import AphasiaDatasetMFCC, AphasiaDatasetSpectrogram, AphasiaDatasetWaveform

from collections import Counter
from models.wav2vecClassifier import Wav2vecClassifier
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain

2025-04-16 00:18:33.223557: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 00:18:33.313815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744751913.351490   18618 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744751913.367236   18618 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 00:18:33.474794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
AUDIO_LENGTH = 6_000
SEQUENCE_LENGTH = 31
MFCC = 128
print(f"It's {DEVICE} time!!!")

It's cuda time!!!


In [3]:
DATA_DIR = os.path.join('..', 'data')
VOICES_DIR = os.path.join(DATA_DIR, 'Voices_wav')
APHASIA_DIR = os.path.join(VOICES_DIR, 'Aphasia')
NORM_DIR = os.path.join(VOICES_DIR, 'Norm')
RIR_DIR = os.path.join(DATA_DIR, 'RIRs')
NOISE_DIR = os.path.join(DATA_DIR, 'noise')

In [4]:
augmentations = audio_augment = Compose([
                AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.02, p=0.7),
                TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
                PitchShift(min_semitones=-3, max_semitones=3, p=0.5),
                Shift(min_shift=-0.3, max_shift=0.3, p=0.5),
                Gain(min_gain_db=-8, max_gain_db=8, p=0.5),
            ])

In [5]:
train_dataset = AphasiaDatasetWaveform(os.path.join(DATA_DIR, "train_filenames_mc_1.csv"), VOICES_DIR, snr=(10, 20), noise_dir=NORM_DIR, rirs_dir=RIR_DIR, target_sample_rate=8_000, file_format="wav", transforms=augmentations)

In [6]:
test_dataset = AphasiaDatasetWaveform(os.path.join(DATA_DIR, "val_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, file_format="wav")

In [7]:
val_dataset = AphasiaDatasetWaveform(os.path.join(DATA_DIR, "test_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, file_format="wav")

In [8]:
# Class balancing for train
train_labels = [label for _, label in train_dataset.data]
class_counts = Counter(train_labels)
if len(class_counts) < 2:
    raise ValueError("One of the classes is missing in the training set")

class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for _, label in train_dataset.data]
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

In [9]:
MAX_LEN = 120_000

In [10]:
def pad_sequence(batch):
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    seq, labels = zip(*batch)
    max_len = max(s.shape[1] for s in seq)
    padded = torch.zeros(len(seq), max_len)
    for i, s in enumerate(seq):
        padded[i, :s.shape[1]] = s[0, :MAX_LEN]
    
    return padded, torch.stack(labels) 

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=4, sampler=train_sampler, collate_fn=pad_sequence, drop_last=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=pad_sequence, drop_last=True, num_workers=6)

In [12]:
def train_model(model, dl_train, dl_val, epochs=1, lr=0.001, device="cpu"):
      
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights.values()))).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, threshold=1e-3)
        
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = []
    for epoch in tqdm(range(epochs), desc="Training model"):
        model.train()
        total_train_loss = 0
        total_val_loss = 0
        
        train_acc = []
        val_acc = []
        for features, target in dl_train:
            features, target = features.to(device), target.to(device)
            optimizer.zero_grad()
            
            output = model(features).logits
            preds = torch.argmax(output, dim=1).cpu().detach().numpy()
            train_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.detach().item()
        avg_train_acc = np.stack(train_acc, axis=0).mean()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_loss_list.append(avg_train_loss)
        train_acc_list.append(avg_train_acc)
                
        model.eval()
        
        with torch.no_grad():
            for features, target in dl_val:
                features, target = features.to(device), target.to(device)
                
                output = model(features).logits
                
                preds = torch.argmax(output, dim=1).cpu().detach().numpy()
                val_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
                loss = criterion(output, target)
                total_val_loss += loss.detach().item()
        
        avg_val_acc = np.stack(val_acc, axis=0).mean()
        avg_val_loss = total_val_loss / len(dl_val)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        
        if scheduler:
            try:
                scheduler.step()
            except:
                scheduler.step(avg_val_loss)
        
        tqdm.write(f"Epoch {epoch}: train loss: {avg_train_loss:.3f}, train balanced acc: {avg_train_acc:.2f}, test loss: {avg_val_loss:.3f}, test balanced acc: {avg_val_acc:.2f}, lr: {optimizer.param_groups[0]['lr']}")

            
    return model, train_loss_list, val_loss_list, train_acc_list, val_acc_list

In [13]:
wav2vec = Wav2vecClassifier(num_labels=4, unfreeze=0.0)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from termcolor import colored
from collections import defaultdict

def beautiful_int(i):
    i = str(i)
    return ".".join(reversed([i[max(j, 0):j+3] for j in range(len(i) - 3, -3, -3)]))

# Calculate the total number of parameters in our model

def model_num_params(model, verbose_all=True, verbose_only_learnable=False):
    sum_params = 0
    sum_learnable_params = 0
    submodules = defaultdict(lambda : [0, 0])
    for name, param in model.named_parameters():
        num_params = np.prod(param.shape)
        if verbose_all or (verbose_only_learnable and param[1].requires_grad):
            print(
                colored(
                    '{: <42} ~  {: <9} params ~ grad: {}'.format(
                        name,
                        beautiful_int(num_params),
                        param.requires_grad,
                    ),
                    {True: "green", False: "red"}[param.requires_grad],
                )
            )
        sum_params += num_params
        sm = name.split(".")[0]
        submodules[sm][0] += num_params
        if param.requires_grad:
            sum_learnable_params += num_params
            submodules[sm][1] += num_params
    print(
        f'\nIn total:\n  - {beautiful_int(sum_params)} params\n  - {beautiful_int(sum_learnable_params)} learnable params'
    )

    for sm, v in submodules.items():
        print(
            f"\n . {sm}:\n .   - {beautiful_int(submodules[sm][0])} params\n .   - {beautiful_int(submodules[sm][1])} learnable params"
        )
    return sum_params, sum_learnable_params


sum_params, sum_learnable_params = model_num_params(wav2vec)

wav2vec.wav2vec2.masked_spec_embed         ~  768       params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.0.conv.weight ~  5.120     params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight ~  512       params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias ~  512       params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.1.conv.weight ~  786.432   params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.2.conv.weight ~  786.432   params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.3.conv.weight ~  786.432   params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.4.conv.weight ~  786.432   params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.5.conv.weight ~  524.288   params ~ grad: True
wav2vec.wav2vec2.feature_extractor.conv_layers.6.conv.weight ~  524.288   params ~ grad: True
wav2vec.wav2vec2.feature_projection.layer_norm.weight ~  512       p

In [15]:
wav2vec, train_l, val_l, train_acc, val_acc = train_model(wav2vec, train_dataloader, val_dataloader, epochs=30, lr=0.001, device=DEVICE)

Training model:   3%|▎         | 1/30 [03:26<1:39:54, 206.72s/it]

Epoch 0: train loss: 1.370, train balanced acc: 0.26, test loss: 1.328, test balanced acc: 0.31, lr: 0.001


Training model:   7%|▋         | 2/30 [06:59<1:38:00, 210.01s/it]

Epoch 1: train loss: 1.362, train balanced acc: 0.26, test loss: 1.326, test balanced acc: 0.31, lr: 0.001


Training model:  10%|█         | 3/30 [10:31<1:35:04, 211.28s/it]

...

Training model:  40%|████      | 12/30 [42:35<1:03:58, 213.24s/it]

Epoch 11: train loss: 1.359, train balanced acc: 0.24, test loss: 1.311, test balanced acc: 0.31, lr: 0.0005


Training model:  43%|████▎     | 13/30 [46:08<1:00:20, 212.96s/it]

Epoch 12: train loss: 1.362, train balanced acc: 0.24, test loss: 1.314, test balanced acc: 0.31, lr: 0.00025


Training model:  43%|████▎     | 13/30 [48:23<1:03:16, 223.33s/it]

KeyboardInterrupt



In [16]:
wav2vec, train_l, val_l, train_acc, val_acc = train_model(wav2vec, train_dataloader, val_dataloader, epochs=30, lr=0.001, device=DEVICE)

Training model:   3%|▎         | 1/30 [02:43<1:19:05, 163.63s/it]

Epoch 0: train loss: 1.349, train balanced acc: 0.25, test loss: 1.294, test balanced acc: 0.31, lr: 0.001


Training model:   7%|▋         | 2/30 [05:28<1:16:42, 164.38s/it]

Epoch 1: train loss: 1.351, train balanced acc: 0.24, test loss: 1.335, test balanced acc: 0.31, lr: 0.001


Training model:  10%|█         | 3/30 [08:14<1:14:21, 165.24s/it]

...

Epoch 27: train loss: 1.060, train balanced acc: 0.45, test loss: 1.034, test balanced acc: 0.48, lr: 3.125e-05


Training model:  97%|█████████▋| 29/30 [1:22:18<02:52, 172.91s/it]

Epoch 28: train loss: 1.036, train balanced acc: 0.46, test loss: 1.065, test balanced acc: 0.50, lr: 3.125e-05


Training model: 100%|██████████| 30/30 [1:25:12<00:00, 170.41s/it]

Epoch 29: train loss: 1.029, train balanced acc: 0.47, test loss: 1.148, test balanced acc: 0.48, lr: 1.5625e-05


In [17]:
wav2vec, train_l, val_l, train_acc, val_acc = train_model(wav2vec, train_dataloader, val_dataloader, epochs=60, lr=1.5625e-05, device=DEVICE)

Training model:   2%|▏         | 1/60 [02:44<2:42:02, 164.78s/it]

Epoch 0: train loss: 1.011, train balanced acc: 0.49, test loss: 1.105, test balanced acc: 0.49, lr: 1.5625e-05


Training model:   3%|▎         | 2/60 [05:32<2:40:57, 166.50s/it]

Epoch 1: train loss: 1.017, train balanced acc: 0.47, test loss: 1.117, test balanced acc: 0.49, lr: 1.5625e-05


Training model:   5%|▌         | 3/60 [08:19<2:38:25, 166.77s/it]

...

Training model:  82%|████████▏ | 49/60 [2:19:29<31:59, 174.52s/it]

Epoch 48: train loss: 0.923, train balanced acc: 0.54, test loss: 1.071, test balanced acc: 0.51, lr: 1.52587890625e-08


Training model:  83%|████████▎ | 50/60 [2:22:25<29:07, 174.77s/it]

Epoch 49: train loss: 0.934, train balanced acc: 0.53, test loss: 1.071, test balanced acc: 0.51, lr: 1.52587890625e-08


Training model:  83%|████████▎ | 50/60 [2:23:44<28:44, 172.49s/it]

KeyboardInterrupt



In [19]:
def test_model(model, test_dataset, max_seq=420):
    model = model.to(DEVICE)
        
    model.eval()
    preds = []
    targets = []
    with torch.no_grad():
        for features, target in tqdm(test_dataset):
            features = features.to(DEVICE)
            target = target.to(DEVICE)
            if model.__class__.__name__ == "Wav2vecClassifier":
                label = model(features).logits.to("cpu").detach().numpy().squeeze()
            elif model.__class__.__name__ == "MobileNet":
                label = model(features[None, ...]).to("cpu").detach().numpy().squeeze()
            else:
                padded_features = torch.zeros(1, features.shape[0], max_seq, device=DEVICE)
                padded_features[0, :, :features.shape[-1]] = features[..., :max_seq]
                label = model(padded_features).to("cpu").detach().numpy().squeeze()
            preds.append(label.argmax(axis=-1))
            targets.append(target.to("cpu").item())

    preds = np.array(preds)
    # print(targets)
    print(classification_report(targets, preds))    

    return preds

In [20]:
test_model(wav2vec, test_dataset)

100%|██████████| 806/806 [00:14<00:00, 54.41it/s]


              precision    recall  f1-score   support

           0       0.80      0.89      0.85       104
           1       0.24      0.07      0.11       185
           2       0.41      0.73      0.52       268
           3       0.60      0.37      0.46       249

    accuracy                           0.49       806
   macro avg       0.51      0.52      0.48       806
weighted avg       0.48      0.49      0.45       806



array([0, 2, 3, 3, 2, 0, 2, 3, 2, 0, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2,
       3, 2, 2, 2, 2, 0, 3, 3, 2, 2, 0, 2, 2, 3, 0, 2, 3, 3, 2, 2, 0, 2,
       2, 3, 2, 2, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 2, 0, 2, 3, 3, 1, 0, 2,
       2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 3, 3, 2, 2, 3, 2,
       2, 2, 2, 0, 2, 0, 0, 2, 0, 3, 2, 2, 0, 2, 2, 2, 3, 2, 0, 0, 0, 3,
       2, 0, 3, 2, 0, 3, 0, 2, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 0, 2, 3,
       2, 1, 1, 0, 2, 1, 1, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0,
       3, 3, 2, 0, 0, 2, 2, 1, 2, 2, 2, 0, 3, 3, 3, 0, 3, 0, 2, 2, 2, 3,
       2, 0, 3, 2, 2, 0, 2, 2, 2, 1, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 3, 3,
       2, 3, 0, 2, 2, 2, 2, 2, 0, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2,
       2, 0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 3, 1,
       2, 2, 2, 2, 2, 1, 2, 0, 3, 0, 3, 3, 2, 2, 2, 2, 0, 2, 0, 3, 0, 0,
       2, 3, 2, 0, 2, 3, 3, 0, 3, 3, 1, 2, 1, 2, 3,

In [21]:
augmentations = Compose([
                AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.02, p=0.5),
                TimeStretch(min_rate=0.3, max_rate=1.9, p=0.5),
                PitchShift(min_semitones=-6, max_semitones=6, p=0.5),
                Shift(min_shift=-0.6, max_shift=0.6, p=0.5),
                Gain(min_gain_db=-14, max_gain_db=14, p=0.5),
            ])

In [22]:
train_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "train_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav", rirs_dir=RIR_DIR, transforms=augmentations)
test_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "val_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav")
val_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "test_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav")

 # Class balancing for train
train_labels = [label for _, label in train_dataset.data]
class_counts = Counter(train_labels)
if len(class_counts) < 2:
    raise ValueError("One of the classes is missing in the training set")


class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for _, label in train_dataset.data]
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

In [23]:
def pad_sequence(batch):
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    seq, labels = zip(*batch)
    max_len = max(s.shape[1] for s in seq)

    padded = torch.zeros(len(seq), MFCC, SEQUENCE_LENGTH)
    for i, s in enumerate(seq):
        padded[i, ..., :s.shape[-1]] = s[..., :SEQUENCE_LENGTH]
    
    return padded, torch.stack(labels) 

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler, collate_fn=pad_sequence, drop_last=True, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=pad_sequence, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=pad_sequence, drop_last=True, num_workers=6)

In [25]:
def train_model(model, dl_train, dl_val, epochs=1, lr=0.001, device="cpu"):
      
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights.values()))).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-3)
        
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, threshold=1e-3)
        
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = []
    for epoch in tqdm(range(epochs), desc="Training model"):
        model.train()
        total_train_loss = 0
        total_val_loss = 0
        
        train_acc = []
        val_acc = []
        for features, target in dl_train:
            features, target = features.to(device), target.to(device)
            optimizer.zero_grad()
            
            output = model(features.unsqueeze(1)).squeeze()

            preds = torch.argmax(output, dim=1).cpu().detach().numpy()
            train_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.detach().item()
        
        avg_train_acc = np.stack(train_acc, axis=0).mean()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_loss_list.append(avg_train_loss)
        train_acc_list.append(avg_train_acc)
                
        model.eval()
        
        with torch.no_grad():
            for features, target in dl_val:
                features, target = features.to(device), target.to(device)
                
                output = model(features.unsqueeze(1)).squeeze()
                
                preds = torch.argmax(output, dim=1).cpu().detach().numpy()
                val_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
                loss = criterion(output, target)
                total_val_loss += loss.detach().item()
        
        avg_val_acc = np.stack(val_acc, axis=0).mean()
        avg_val_loss = total_val_loss / len(dl_val)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        
        if scheduler:
            try:
                scheduler.step()
            except:
                scheduler.step(avg_val_loss)
        
        tqdm.write(f"Epoch {epoch}: train loss: {avg_train_loss:.3f}, train balanced acc: {avg_train_acc:.2f}, test loss: {avg_val_loss:.3f}, test balanced acc: {avg_val_acc:.2f}, lr: {optimizer.param_groups[0]['lr']}")

            
    return model, train_loss_list, val_loss_list, train_acc_list, val_acc_list

In [26]:
from models.cnn import MobileNet

cnn = MobileNet(num_classes=4)

In [27]:
cnn, train_l, val_l, train_accuracy, val_accuracy = train_model(cnn, train_dataloader, val_dataloader, epochs=10, lr=1e-4, device=DEVICE)

Training model:  10%|█         | 1/10 [00:02<00:21,  2.41s/it]

Epoch 0: train loss: 1.352, train balanced acc: 0.24, test loss: 1.366, test balanced acc: 0.31, lr: 0.0001


Training model:  20%|██        | 2/10 [00:04<00:19,  2.39s/it]

Epoch 1: train loss: 1.269, train balanced acc: 0.33, test loss: 1.331, test balanced acc: 0.31, lr: 0.0001


Training model:  30%|███       | 3/10 [00:07<00:16,  2.40s/it]

...

Epoch 7: train loss: 0.617, train balanced acc: 0.74, test loss: 1.815, test balanced acc: 0.31, lr: 5e-05


Training model:  90%|█████████ | 9/10 [00:21<00:02,  2.43s/it]

Epoch 8: train loss: 0.509, train balanced acc: 0.79, test loss: 1.879, test balanced acc: 0.31, lr: 5e-05


Training model: 100%|██████████| 10/10 [00:24<00:00,  2.42s/it]

Epoch 9: train loss: 0.393, train balanced acc: 0.84, test loss: 2.085, test balanced acc: 0.31, lr: 5e-05


In [28]:
test_model(cnn, test_dataset)

100%|██████████| 807/807 [00:01<00:00, 436.07it/s]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       106
           1       0.08      0.01      0.01       186
           2       0.00      0.00      0.00       270
           3       0.30      0.97      0.46       245

    accuracy                           0.29       807
   macro avg       0.09      0.24      0.12       807
weighted avg       0.11      0.29      0.14       807



array([3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [29]:
train_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "train_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav", transforms=augmentations)#  noise_dir=NORM_DIR, rirs_dir=RIR_DIR, snr=(-5, 20))
test_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "val_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav")
val_dataset = AphasiaDatasetMFCC(os.path.join(DATA_DIR, "test_filenames_mc_1.csv"), VOICES_DIR, target_sample_rate=8_000, mfcc=MFCC, n_mels=128, fft_size=512,
                 hop_length=256, win_length=512, min_duration=10, max_duration=15, file_format="wav")
# Class balancing for train
train_labels = [label for _, label in train_dataset.data]
class_counts = Counter(train_labels)
if len(class_counts) < 2:
    raise ValueError("One of the classes is missing in the training set")


class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for _, label in train_dataset.data]
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

In [30]:
def pad_sequence(batch):
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    seq, labels = zip(*batch)
    max_len = max(s.shape[1] for s in seq)
    padded = torch.zeros(len(seq), MFCC, SEQUENCE_LENGTH)
    for i, s in enumerate(seq):
        padded[i, ..., :s.shape[-1]] = s[..., :SEQUENCE_LENGTH]
    
    return padded, torch.stack(labels) 

In [31]:
train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, collate_fn=pad_sequence, drop_last=True, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=pad_sequence, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=pad_sequence, drop_last=True, num_workers=6)

In [32]:
def train_model(model, dl_train, dl_val, epochs=1, lr=0.001, device="cpu"):
      
    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-3)
        
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, threshold=1e-3)
        
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = []
    for epoch in tqdm(range(epochs), desc="Training model"):
        model.train()
        total_train_loss = 0
        total_val_loss = 0
        
        train_acc = []
        val_acc = []
        for features, target in dl_train:
            features, target = features.to(device), target.to(device)
            optimizer.zero_grad()
            
            output = model(features).squeeze()
            preds = torch.argmax(output, dim=1).cpu().detach().numpy()
            train_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.detach().item()
        avg_train_acc = np.stack(train_acc, axis=0).mean()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_loss_list.append(avg_train_loss)
        train_acc_list.append(avg_train_acc)
                
        model.eval()
        
        with torch.no_grad():
            for features, target in dl_val:
                features, target = features.to(device), target.to(device)
                
                output = model(features).squeeze()
                
                preds = torch.argmax(output, dim=1).cpu().detach().numpy()
                val_acc.append(accuracy_score(target.cpu().detach().numpy(), preds))
                loss = criterion(output, target)
                total_val_loss += loss.detach().item()
        
        avg_val_acc = np.stack(val_acc, axis=0).mean()
        avg_val_loss = total_val_loss / len(dl_val)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        
        if scheduler:
            try:
                scheduler.step()
            except:
                scheduler.step(avg_val_loss)
        
        tqdm.write(f"Epoch {epoch}: train loss: {avg_train_loss:.3f}, train balanced acc: {avg_train_acc:.2f}, test loss: {avg_val_loss:.3f}, test balanced acc: {avg_val_acc:.2f}, lr: {optimizer.param_groups[0]['lr']}")

            
    return model, train_loss_list, val_loss_list, train_acc_list, val_acc_list

In [33]:
from models.swishnet import SwishNet

swishnet = SwishNet(MFCC, 4, input_size=SEQUENCE_LENGTH, dropout_rate=0.55)

In [34]:
swishnet, train_l, val_l, train_accuracy, val_accuracy = train_model(swishnet, train_dataloader, val_dataloader, epochs=100, lr=1e-4, device=DEVICE)

Training model:   1%|          | 1/100 [00:00<01:08,  1.45it/s]

Epoch 0: train loss: 1.384, train balanced acc: 0.28, test loss: 1.384, test balanced acc: 0.31, lr: 0.0001


Training model:   2%|▏         | 2/100 [00:01<01:13,  1.33it/s]

Epoch 1: train loss: 1.383, train balanced acc: 0.28, test loss: 1.383, test balanced acc: 0.32, lr: 0.0001


Training model:   3%|▎         | 3/100 [00:02<01:10,  1.37it/s]

...

Epoch 97: train loss: 1.280, train balanced acc: 0.41, test loss: 1.360, test balanced acc: 0.33, lr: 1.220703125e-08


Training model:  99%|█████████▉| 99/100 [01:02<00:00,  1.58it/s]

Epoch 98: train loss: 1.275, train balanced acc: 0.40, test loss: 1.361, test balanced acc: 0.33, lr: 1.220703125e-08


Training model: 100%|██████████| 100/100 [01:03<00:00,  1.58it/s]

Epoch 99: train loss: 1.278, train balanced acc: 0.41, test loss: 1.362, test balanced acc: 0.33, lr: 1.220703125e-08
